In [2]:
import os
import numpy as np
from scipy.io import loadmat
from sklearn.model_selection import train_test_split


In [3]:
data_dir = "D:/mohan/vibration/cwru/raw"  # Update this to your actual directory path
segment_length = 2048
test_size = 0.2
val_size = 0.1
random_state = 42

In [4]:
def get_label_from_filename(fname):
    """Assign labels based on filename conventions.
    Adjust logic if naming conventions differ."""
    fname = fname.lower()
    if "normal" in fname:
        return 0  # Normal condition
    elif fname.startswith("b"):
        return 1  # Ball fault
    elif fname.startswith("ir"):
        return 2  # Inner Race fault
    elif fname.startswith("or"):
        return 3  # Outer Race fault
    else:
        raise ValueError(f"Unknown fault type in filename {fname}")

In [5]:
import os
import glob
import re
import numpy as np
from scipy.io import loadmat
from sklearn.model_selection import train_test_split

# -----------------------
# Configuration
# -----------------------
data_dir = "D:/mohan/vibration/cwru/raw/"
segment_length = 2048
test_size = 0.2
val_size = 0.1
random_state = 42

def get_label_from_filename(fname):
    fname = fname.lower()
    if "normal" in fname:
        return 0  # Normal condition
    elif fname.startswith("b"):
        return 1  # Ball fault
    elif fname.startswith("ir"):
        return 2  # Inner Race fault
    elif fname.startswith("or"):
        return 3  # Outer Race fault
    else:
        raise ValueError(f"Unknown fault type in filename: {fname}")

# -----------------------
# Load and Segment Data from All Files
# -----------------------
segments = []
labels = []

mat_files = glob.glob(os.path.join(data_dir, "*.mat"))
if not mat_files:
    raise FileNotFoundError(f"No .mat files found in {data_dir}")

for filepath in mat_files:
    fname = os.path.basename(filepath)
    label = get_label_from_filename(fname)
    
    # Load the MATLAB file
    mat_data = loadmat(filepath)
    # Identify the DE_time variable
    possible_keys = [k for k in mat_data.keys() if k.endswith("_DE_time")]
    if not possible_keys:
        raise ValueError(f"No DE_time variable found in {fname}")
    # Assuming one DE_time variable per file
    chosen_key = possible_keys[0]
    
    # Extract the vibration signal (DE_time)
    signal = mat_data[chosen_key].ravel()

    # Segment the signal
    num_samples = len(signal)
    for start in range(0, num_samples - segment_length + 1, segment_length):
        segment = signal[start:start+segment_length]
        segments.append(segment)
        labels.append(label)

segments = np.array(segments)
labels = np.array(labels)

print(f"Total segments: {segments.shape[0]}")
print("Label distribution:", {lbl: np.sum(labels == lbl) for lbl in np.unique(labels)})

# -----------------------
# Normalization
# -----------------------
mean = np.mean(segments)
std = np.std(segments)
segments = (segments - mean) / (std + 1e-8)

# -----------------------
# Train/Val/Test Split
# -----------------------
# First split into train+val and test
X_trainval, X_test, y_trainval, y_test = train_test_split(
    segments, labels, test_size=test_size, random_state=random_state, stratify=labels
)

val_fraction = val_size / (1 - test_size)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=val_fraction, random_state=random_state, stratify=y_trainval
)

print("Train set:", X_train.shape, y_train.shape)
print("Val set:", X_val.shape, y_val.shape)
print("Test set:", X_test.shape, y_test.shape)

# After this step, you have:
# X_train, y_train, X_val, y_val, X_test, y_test
# ready for Step 4 (patch creation for the Transformer model).


Total segments: 2369
Label distribution: {np.int64(0): np.int64(236), np.int64(1): np.int64(711), np.int64(2): np.int64(711), np.int64(3): np.int64(711)}
Train set: (1658, 2048) (1658,)
Val set: (237, 2048) (237,)
Test set: (474, 2048) (474,)


In [6]:


# Assuming we have X_train, X_val, X_test from Step 3
# and segment_length is known (e.g., 2048).
# Decide on a patch size
patch_size = 32
assert segment_length % patch_size == 0, "segment_length must be divisible by patch_size"
num_patches = segment_length // patch_size

def create_patches(X, patch_size):
    # X is of shape (N, segment_length)
    N, L = X.shape
    # Reshape to (N, num_patches, patch_size)
    X_patched = X.reshape(N, num_patches, patch_size)
    return X_patched

# Apply to all splits
X_train_patched = create_patches(X_train, patch_size)
X_val_patched = create_patches(X_val, patch_size)
X_test_patched = create_patches(X_test, patch_size)

print("X_train_patched shape:", X_train_patched.shape)
print("X_val_patched shape:", X_val_patched.shape)
print("X_test_patched shape:", X_test_patched.shape)

# y_train, y_val, y_test stay the same
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)
print("y_test shape:", y_test.shape)


X_train_patched shape: (1658, 64, 32)
X_val_patched shape: (237, 64, 32)
X_test_patched shape: (474, 64, 32)
y_train shape: (1658,)
y_val shape: (237,)
y_test shape: (474,)


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

# -----------------------
# User Configuration
# -----------------------
num_epochs = 100
batch_size = 32
learning_rate = 1e-3
num_classes = 4  # Normal, Ball b, Inner Race, Outer Race

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Assume we have X_train_patched, y_train, X_val_patched, y_val, X_test_patched, y_test as NumPy arrays
# Convert them to PyTorch tensors
X_train_tensor = torch.tensor(X_train_patched, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val_patched, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_patched, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# -----------------------
# Model Definition
# -----------------------
# Pseudocode / Example:
# If you have a PatchTST model class, load it here. For demonstration, let's define a generic model:
# Replace `YourPatchTSTModelClass` with the actual class name when using a real model.

class YourPatchTSTModelClass(nn.Module):
    def __init__(self, d_model=64, num_layers=2, num_heads=4, num_patches=64, patch_size=32):
        super().__init__()
        # This is a placeholder. In practice, you would initialize the PatchTST model components here.
        # For demonstration, we use a simple transformer encoder and global pooling:
        
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # Classification head will be added after we get the transformed features
        self.classifier = nn.Linear(d_model, num_classes)
        
        # Positional encoding or other embeddings would be defined here as needed

    def forward(self, x):
        # x shape: (batch, num_patches, patch_size)
        # For a real PatchTST, you would have patch embedding + positional encoding steps here.
        
        # Let's assume we simply flatten patch dimension to feed into a linear layer:
        # In a real scenario, you'd have embeddings and feed into the transformer properly.
        
        # For demonstration, we'll treat each patch as a token and feed directly:
        batch_size, num_patches, patch_size = x.size()
        
        # Simple linear projection to d_model
        d_model = 64
        projection = nn.Linear(patch_size, d_model).to(x.device)
        x = projection(x)  # shape: (batch, num_patches, d_model)
        
        # Transformer expects (seq_len, batch, embedding_dim)
        x = x.permute(1, 0, 2)  # (num_patches, batch, d_model)
        
        # Pass through transformer
        encoded = self.transformer(x)  # (num_patches, batch, d_model)
        
        # Global average pooling over patches
        encoded = encoded.mean(dim=0)  # (batch, d_model)
        
        # Classification
        logits = self.classifier(encoded)  # (batch, num_classes)
        return logits

model = YourPatchTSTModelClass().to(device)

# -----------------------
# Training Setup
# -----------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

# -----------------------
# Training Loop
# -----------------------
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    train_correct = 0
    total = 0
    
    for X_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)  
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * X_batch.size(0)
        _, preds = outputs.max(1)
        train_correct += preds.eq(y_batch).sum().item()
        total += y_batch.size(0)
    
    train_loss /= total
    train_acc = train_correct / total
    
    # Validation
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for X_val_batch, y_val_batch in val_loader:
            X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
            val_outputs = model(X_val_batch)
            v_loss = criterion(val_outputs, y_val_batch)
            val_loss += v_loss.item() * X_val_batch.size(0)
            _, val_preds = val_outputs.max(1)
            val_correct += val_preds.eq(y_val_batch).sum().item()
            val_total += y_val_batch.size(0)
    
    val_loss /= val_total
    val_acc = val_correct / val_total
    
    print(f"Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

# -----------------------
# Testing
# -----------------------
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for X_test_batch, y_test_batch in test_loader:
        X_test_batch, y_test_batch = X_test_batch.to(device), y_test_batch.to(device)
        test_outputs = model(X_test_batch)
        _, test_preds = test_outputs.max(1)
        test_correct += test_preds.eq(y_test_batch).sum().item()
        test_total += y_test_batch.size(0)

test_acc = test_correct / test_total
print(f"Test Accuracy: {test_acc:.4f}")


d:\mohan\code\cwru_code\lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1: Train Loss: 1.3575, Train Acc: 0.2998, Val Loss: 1.3053, Val Acc: 0.4008


Epoch 2: Train Loss: 1.2918, Train Acc: 0.3667, Val Loss: 1.2073, Val Acc: 0.4093


Epoch 3: Train Loss: 1.1442, Train Acc: 0.4234, Val Loss: 1.1360, Val Acc: 0.3924


Epoch 4: Train Loss: 0.9956, Train Acc: 0.5265, Val Loss: 0.8787, Val Acc: 0.5612


...

Epoch 96: Train Loss: 0.4854, Train Acc: 0.7805, Val Loss: 0.4743, Val Acc: 0.8270


Epoch 97: Train Loss: 0.5284, Train Acc: 0.7726, Val Loss: 0.4311, Val Acc: 0.8101


Epoch 98: Train Loss: 0.4965, Train Acc: 0.7901, Val Loss: 0.4173, Val Acc: 0.8397


Epoch 99: Train Loss: 0.4566, Train Acc: 0.7937, Val Loss: 0.4024, Val Acc: 0.8017


Epoch 100: Train Loss: 0.4585, Train Acc: 0.8040, Val Loss: 0.4497, Val Acc: 0.7932
Test Accuracy: 0.7869


In [8]:
import torch
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from torch.utils.data import TensorDataset, DataLoader

# Assuming these variables are available from previous steps:
# X_test_patched: shape (N_test, num_patches, patch_size)
# y_test: shape (N_test,)
# model: trained model from previous steps

batch_size = 32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Convert test data to tensors and create a DataLoader
X_test_tensor = torch.tensor(X_test_patched, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model.eval()
model.to(device)

all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        outputs = model(X_batch)  # shape: (batch, num_classes)
        _, preds = torch.max(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(y_batch.cpu().numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Compute confusion matrix
cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:")
print(cm)

# Compute classification report
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, digits=4))


Confusion Matrix:
[[ 47   0   0   0]
 [  1 117  19   5]
 [  0   7 126   9]
 [  0  51   3  89]]

Classification Report:
              precision    recall  f1-score   support

           0     0.9792    1.0000    0.9895        47
           1     0.6686    0.8239    0.7382       142
           2     0.8514    0.8873    0.8690       142
           3     0.8641    0.6224    0.7236       143

    accuracy                         0.7996       474
   macro avg     0.8408    0.8334    0.8300       474
weighted avg     0.8131    0.7996    0.7979       474

